# Chargement du modèle GoogLeNet pré-entraîné

In [5]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.googlenet(pretrained=True)


c:\Users\bouch\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bouch\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Gel des paramètres d'un modèle

In [6]:
for param in model.parameters():
    param.requires_grad = False

### Chargement et préparation des jeux de données avec PyTorch

In [7]:
import torch
import torchvision
from torch.utils.data import DataLoader

train_dataset = torch.load("../data/Data_Sets/train_dataset.pt", weights_only=False)
val_dataset   = torch.load("../data/Data_Sets/val_dataset.pt", weights_only=False)
test_dataset  = torch.load("../data/Data_Sets/test_dataset.pt", weights_only=False)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


2808
486
487


# Remplacement du classificateur par un réseau personnalisé

In [8]:
num_classes = len(train_dataset.classes)

model.fc = nn.Sequential(
    nn.Linear(1024, 256), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes),
)

# model.fc = nn.Sequential(
#     nn.Linear(1024, 512), 
#     nn.ReLU(),
#     nn.Dropout(0.3),
#     nn.Linear(512, 256), 
#     nn.ReLU(),
#     nn.Dropout(0.5),
#   nn.Linear(256, 128), 
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(128, num_classes)
# )

In [9]:
# print(model)

# Configuration de l'entraînement du modèle

In [10]:
# Determiner le learning rate et loss function et aussi l’optomizer pour le modele
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
learning_rate  = 0.001   
loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)



cuda


# Entraînement et évaluation du modèle

In [11]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train() 
    total_loss = 0.0         
    total_correct = 0         
    total_samples = 0 

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()
        total_samples += labels.size(0)
        
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * total_correct / total_samples
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch 1/5 - Loss: 0.5323, Accuracy: 81.70%
Epoch 2/5 - Loss: 0.2246, Accuracy: 92.81%
Epoch 3/5 - Loss: 0.1636, Accuracy: 94.62%
Epoch 4/5 - Loss: 0.1438, Accuracy: 95.23%
Epoch 5/5 - Loss: 0.1076, Accuracy: 96.19%


### Évaluation du modèle : calcul de la perte et de la précision

In [14]:
# Évaluation et test
def evaluate(loader):
    model.eval()
    correct, total, loss_total = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss_total += loss_function(outputs, labels).item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return loss_total / len(loader), 100 * correct / total

val_loss, val_acc = evaluate(val_loader)
print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

test_loss, test_acc = evaluate(test_loader)
print(f"Test - Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%")


Validation - Loss: 0.0577, Accuracy: 98.15%
Test - Loss: 0.0688, Accuracy: 97.54%


### Sauvegarde du modèle entraîné avec PyTorch

In [ ]:
torch.save(model, "../data\googlenet_fc_complete.pth")